# Scaping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
import csv
from datetime import datetime
import logging


In [2]:
list_of_urls = ["https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
"https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2"]

# First 50 pages of Ipad 2021 reviews
for i in list(range(3,51)):
    j = "https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_"+str(i)+"?ie=UTF8&reviewerType=all_reviews&pageNumber="+str(i)
    list_of_urls.append(j)



In [3]:
headers = {
    "authority": "www.amazon.com",
    "pragma": "no-cache",
    "cache-control": "no-cache",
    "dnt": "1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "sec-fetch-site": "none",
    "sec-fetch-mode": "navigate",
    "sec-fetch-dest": "document",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
}



def get_page_html(page_url: str) -> str:
    params = {'api_key': "0b23a0f5a41058eee1b906100a0f1ae7", 'url': page_url}
    resp = requests.get('http://api.scraperapi.com/', params=urlencode(params), headers=headers)
    return resp.text



def get_reviews_from_html(page_html: str) -> BeautifulSoup:
    soup = BeautifulSoup(page_html, "lxml")
    reviews = soup.find_all("div", {"class": "a-section celwidget"})
    return reviews


def get_review_date(soup_object: BeautifulSoup):
    date_string = soup_object.find("span", {"class": "review-date"}).get_text()
    return date_string


def get_review_text(soup_object: BeautifulSoup) -> str:
    review_text = soup_object.find(
        "span", {"class": "a-size-base review-text review-text-content"}
    ).get_text()
    return review_text.strip()


def get_review_header(soup_object: BeautifulSoup) -> str:
    review_header = soup_object.find(
        "a",
        {
            "class": "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"
        },
    ).get_text()
    return review_header.strip()


def get_number_stars(soup_object: BeautifulSoup) -> str:
    stars = soup_object.find("span", {"class": "a-icon-alt"}).get_text()
    return stars.strip()



def orchestrate_data_gathering(single_review: BeautifulSoup) -> dict:
    return {
        "review_text": get_review_text(single_review),
        "review_date": get_review_date(single_review),
        "review_title": get_review_header(single_review),
        "review_stars": get_number_stars(single_review),
    }


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls[0:10]:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_1 = pd.DataFrame.from_records(all_results)
    out_1.to_csv('0_10.csv')
    
out_1

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=5
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_6?ie=UTF8&reviewerType=al

,review_text,review_date,review_title,review_stars
0,These ipads come with the A bionic chip so the...,"Reviewed in the United States on March 17, 2023",Great value for new,5.0 out of 5 stars
1,My wife upgraded from her iPad Air 2. She play...,"Reviewed in the United States on March 9, 2023",The perfect upgrade from iPad Air 2!,5.0 out of 5 stars
2,I've move from my 9 year old iPad air 2 to thi...,"Reviewed in the United States on March 14, 2023",Love my iPad,5.0 out of 5 stars
3,"This is my third iPad, and I couldn't be happi...","Reviewed in the United States on March 15, 2023","Too big for an eye pad, but great for everythi...",5.0 out of 5 stars
4,Hats off to Apple. This iPad doesn’t disappoin...,"Reviewed in the United States on March 20, 2023",Works well and easy setup,5.0 out of 5 stars
...,...,...,...,...
85,Mi hija adolescente está fascinada con este di...,"Reviewed in the United States on January 4, 2023",Una opción perfecta si sueñas tener una iPad c...,5.0 out of 5 stars
86,👨‍💻,"Reviewed in the United States on February 17, ...",🍎,5.0 out of 5 stars
87,I purchased an iPad for my computer illiterate...,"Reviewed in the United States on December 29, ...",Easy to learn computers with an iPad!,5.0 out of 5 stars
88,Bought this for a xmas gift. Great price and ...,"Reviewed in the United States on January 11, 2023",Apple Ipad,5.0 out of 5 stars


In [4]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls[10:20]:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_2 = pd.DataFrame.from_records(all_results)
    out_2.to_csv('10_20.csv')
    
out_2

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_11?ie=UTF8&reviewerType=all_reviews&pageNumber=11
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_12?ie=UTF8&reviewerType=all_reviews&pageNumber=12
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_13?ie=UTF8&reviewerType=all_reviews&pageNumber=13
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_14?ie=UTF8&reviewerType=all_reviews&pageNumber=14
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_15?ie=UTF8&reviewerType=all_reviews&pageNumber=15
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm

,review_text,review_date,review_title,review_stars
0,So I have 1 problem with this new 9th gen… the...,"Reviewed in the United States on November 27, ...",Best part is 64gb option,5.0 out of 5 stars
1,Got this to replace an older ipad from at leas...,"Reviewed in the United States on January 20, 2023",much improved ipad,5.0 out of 5 stars
2,Initially was hesistent on buying a used produ...,"Reviewed in the United States on January 8, 2023",Good product,4.0 out of 5 stars
3,It’s perfect!,"Reviewed in the United States on February 12, ...",My Christmas gift,5.0 out of 5 stars
4,I can't live without an Ipad. I love them. T...,"Reviewed in the United States on January 21, 2023",Can't live without an Ipad,5.0 out of 5 stars
...,...,...,...,...
85,"Was looking to replace older iPad, needed more...","Reviewed in the United States 🇺🇸 on January 8,...",Good ipad,5.0 out of 5 stars
86,"I bought the 256GB model ""used"" from Amazon Wa...","Reviewed in the United States 🇺🇸 on May 18, 2022",Consider a Used Unit,5.0 out of 5 stars
87,Right from the box ready to go. Personal setti...,Reviewed in the United States 🇺🇸 on December 3...,Great --- as expected,5.0 out of 5 stars
88,This is a great tablet for everyday use. The s...,Reviewed in the United States 🇺🇸 on December 1...,Apple 10.2 inch Ipad 2021,5.0 out of 5 stars


In [5]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls[20:30]:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_3 = pd.DataFrame.from_records(all_results)
    out_3.to_csv('20_30.csv')
    
out_3

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_21?ie=UTF8&reviewerType=all_reviews&pageNumber=21
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_22?ie=UTF8&reviewerType=all_reviews&pageNumber=22
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_23?ie=UTF8&reviewerType=all_reviews&pageNumber=23
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_24?ie=UTF8&reviewerType=all_reviews&pageNumber=24
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_25?ie=UTF8&reviewerType=all_reviews&pageNumber=25
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm

,review_text,review_date,review_title,review_stars
0,My husband’s first iPad and he loves it. Its v...,"Reviewed in the United States 🇺🇸 on January 1,...",My husband loves it!,5.0 out of 5 stars
1,My kid loves their iPad. Seems to work good.,Reviewed in the United States 🇺🇸 on January 13...,So far so good,5.0 out of 5 stars
2,"I'm not an apple user, but this iPad was fair...",Reviewed in the United States 🇺🇸 on December 1...,Wasn't an apple person,5.0 out of 5 stars
3,Super deal everything works super fast shipping,Reviewed in the United States 🇺🇸 on January 13...,Great Deal,5.0 out of 5 stars
4,My wife uses the iPad to read the digital news...,"Reviewed in the United States 🇺🇸 on January 9,...",9th generation ipad.Very good iPad for the money!,5.0 out of 5 stars
...,...,...,...,...
95,If you want a thorough specs review of this 9t...,Reviewed in the United States 🇺🇸 on December 2...,Authentic Apple Product,5.0 out of 5 stars
96,Overall quite happy with the performance and q...,Reviewed in the United States 🇺🇸 on December 2...,Excellent IPad for the price,5.0 out of 5 stars
97,Gen 9. Original packaging. Brand new. Correct ...,Reviewed in the United States 🇺🇸 on September ...,I personally recommend. This was a great optio...,5.0 out of 5 stars
98,If one wants to get into an Apple product that...,Reviewed in the United States 🇺🇸 on November 1...,Solid Value,5.0 out of 5 stars


In [6]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls[30:40]:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_4 = pd.DataFrame.from_records(all_results)
    out_4.to_csv('30_40.csv')
    
out_4

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_31?ie=UTF8&reviewerType=all_reviews&pageNumber=31
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_32?ie=UTF8&reviewerType=all_reviews&pageNumber=32
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_33?ie=UTF8&reviewerType=all_reviews&pageNumber=33
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_34?ie=UTF8&reviewerType=all_reviews&pageNumber=34
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_35?ie=UTF8&reviewerType=all_reviews&pageNumber=35
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm

,review_text,review_date,review_title,review_stars
0,I get this Ipod for my wife and she love it.,Reviewed in the United States 🇺🇸 on December 3...,My wife love it,5.0 out of 5 stars
1,"I thought I was buying refurbed Ipad, which wa...",Reviewed in the United States 🇺🇸 on November 2...,Dry Pleased,5.0 out of 5 stars
2,Have one just like it. Purchased as a Christma...,Reviewed in the United States 🇺🇸 on December 3...,The best,5.0 out of 5 stars
3,IPad has great resolution and easy to use,Reviewed in the United States 🇺🇸 on December 2...,Easy setup. Took less than 5 min,5.0 out of 5 stars
4,"My most recent iPad was lost in an airport, an...",Reviewed in the United States 🇺🇸 on November 9...,Works even better than my last one!,5.0 out of 5 stars
...,...,...,...,...
95,So far so good! Purchased as a birthday gift f...,Reviewed in the United States 🇺🇸 on November 1...,Happy Birthday gift!,5.0 out of 5 stars
96,I was very hesitant to buy this iPad because I...,Reviewed in the United States 🇺🇸 on November 1...,College Student,5.0 out of 5 stars
97,For being brand new it was a good deal!,Reviewed in the United States 🇺🇸 on December 1...,Good price,5.0 out of 5 stars
98,Reviewing this product roughly one year after ...,Reviewed in the United States 🇺🇸 on November 2...,Full year later still working good as new,5.0 out of 5 stars


In [7]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls[40:51]:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_5 = pd.DataFrame.from_records(all_results)
    out_5.to_csv('40_50.csv')
    
out_5

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_41?ie=UTF8&reviewerType=all_reviews&pageNumber=41
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_42?ie=UTF8&reviewerType=all_reviews&pageNumber=42
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_43?ie=UTF8&reviewerType=all_reviews&pageNumber=43
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_44?ie=UTF8&reviewerType=all_reviews&pageNumber=44
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_45?ie=UTF8&reviewerType=all_reviews&pageNumber=45
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm

,review_text,review_date,review_title,review_stars
0,It works,"Reviewed in the United States on December 22, ...",Good,5.0 out of 5 stars
1,I’m not actually going to comment on any of th...,"Reviewed in the United States on July 20, 2022",Just what I needed!,5.0 out of 5 stars
2,I love this Ipad. Very convenient for studying,"Reviewed in the United States on December 20, ...",2021 Apple Ipad,5.0 out of 5 stars
3,Bought this mainly for reading comics and it h...,"Reviewed in the United States on October 28, 2022",Great!,5.0 out of 5 stars
4,"The 2021 Apple 10.2-inch iPad (Wi-Fi, 256GB) -...","Reviewed in the United States on December 1, 2022",Exactly what I expected,5.0 out of 5 stars
...,...,...,...,...
85,Was very happy with business,Reviewed in the United States 🇺🇸 on December 9...,Everything Great!,5.0 out of 5 stars
86,Very happy with the purchase.,Reviewed in the United States 🇺🇸 on December 1...,Best price on the internet for this iPad.,5.0 out of 5 stars
87,met my expectiations,Reviewed in the United States 🇺🇸 on December 1...,all good and bright and beautiful,5.0 out of 5 stars
88,Really like it,Reviewed in the United States 🇺🇸 on December 1...,Exellent!,5.0 out of 5 stars


In [22]:
out_p = pd.concat([out_1,out_2,out_3,out_4,out_5])

In [26]:
out_p['ratings'] = [i[0] for i in out_p['review_stars']]
out_p['reviews'] = out_p['review_text'] + out_p['review_title']
out_p = out_p[['ratings','reviews']]

In [27]:
out_p.to_csv('p.csv')

## Negative

In [15]:
list_of_urls_n = ["https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_unknown?ie=UTF8&reviewerType=all_reviews&pageNumber=1&filterByStar=one_star",
"https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2&filterByStar=one_star"]

# First 50 pages of Ipad 2021 reviews
for i in list(range(3,11)):
    j = "https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_"+str(i)+"?ie=UTF8&reviewerType=all_reviews&pageNumber="+str(i)+"&filterByStar=one_star"
    list_of_urls_n.append(j)

In [16]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    all_results = []

    for u in list_of_urls_n:
        logging.info(u)
        html = get_page_html(u)
        reviews = get_reviews_from_html(html)
        for rev in reviews:
            data = orchestrate_data_gathering(rev)
            all_results.append(data)

    out_n = pd.DataFrame.from_records(all_results)
    out_n.to_csv('n.csv')
    
out_n

INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_unknown?ie=UTF8&reviewerType=all_reviews&pageNumber=1&filterByStar=one_star
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2&filterByStar=one_star
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3&filterByStar=one_star
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4&filterByStar=one_star
INFO:root:https://www.amazon.com/2021-Apple-10-2-inch-Wi-Fi-256GB/product-reviews/B09G96TFF7/ref=cm_cr_getr_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=5&filterByStar=one_star
INFO:root:https://www.amazon.com/2021

,review_text,review_date,review_title,review_stars
0,Bought this as a Santa gift for our son. Deliv...,"Reviewed in the United States 🇺🇸 on March 3, 2023",Doesn't work.,1.0 out of 5 stars
1,Received a new Apple iPad to replace an older ...,Reviewed in the United States 🇺🇸 on October 15...,Apple Products Going Down Hill,1.0 out of 5 stars
2,Received this iPad and 3 days later it wouldn'...,"Reviewed in the United States 🇺🇸 on August 14,...",POS,1.0 out of 5 stars
3,Paid $300 for an iPad for it to quit working d...,Reviewed in the United States 🇺🇸 on February 2...,Quit Working,1.0 out of 5 stars
4,My granddaughter dropped her Apple Ipad on Chr...,Reviewed in the United States 🇺🇸 on December 2...,Apple Warranty Purchase A joke,1.0 out of 5 stars
...,...,...,...,...
95,I do not like that this iPad will not retain e...,"Reviewed in the United States on August 28, 2022",This iPad will not hold news max or kindle dow...,1.0 out of 5 stars
96,Said too many apple ids on this device when I ...,"Reviewed in the United States on May 7, 2022",The one I received was not new.,1.0 out of 5 stars
97,Has already started freezing and not respondin...,"Reviewed in the United States on July 3, 2022",Stopped working after a few months,1.0 out of 5 stars
98,It was broke when I received it,"Reviewed in the United States on October 6, 2022",A broken iPad,1.0 out of 5 stars


In [28]:
out_n['ratings'] = [i[0] for i in out_n['review_stars']]
out_n['reviews'] = out_n['review_text'] + out_n['review_title']
out_n = out_n[['ratings','reviews']]

In [29]:
out_n.to_csv('n.csv')

## Concat

In [17]:
out = pd.concat([out_1,out_2,out_3,out_4,out_5,out_n])

In [19]:
out['ratings'] = [i[0] for i in out['review_stars']]
out['reviews'] = out['review_text'] + out['review_title']

In [20]:
out = out[['ratings','reviews']]

In [21]:
out

,ratings,reviews
0,5,These ipads come with the A bionic chip so the...
1,5,My wife upgraded from her iPad Air 2. She play...
2,5,I've move from my 9 year old iPad air 2 to thi...
3,5,"This is my third iPad, and I couldn't be happi..."
4,5,Hats off to Apple. This iPad doesn’t disappoin...
...,...,...
95,1,I do not like that this iPad will not retain e...
96,1,Said too many apple ids on this device when I ...
97,1,Has already started freezing and not respondin...
98,1,It was broke when I received itA broken iPad
